In [1]:
import os
import pandas as pd
from datetime import datetime
import gc

# Configurações
input_dir = "/home/amcjr/pysus/"
output_dir = "/home/amcjr/pysus/csv_output/"
log_file = os.path.join(output_dir, f"log_processo_{datetime.now().strftime('%d%m%y_%H%M%S')}.xlsx")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def listar_arquivos_parquet(input_dir):
    return [f for f in os.listdir(input_dir) if f.endswith('.parquet')]

def processar_arquivo_parquet(parquet_file):
    try:
        # Extrair informações do nome do arquivo
        ano = "20" + parquet_file[4:6]
        mes = parquet_file[6:8]

        # Carregar dados do arquivo Parquet
        df = pd.read_parquet(os.path.join(input_dir, parquet_file))
        
        # Verificar e garantir que as colunas PA_QTDAPR e PA_VALAPR sejam numéricas
        df['PA_QTDAPR'] = pd.to_numeric(df['PA_QTDAPR'], errors='coerce').fillna(0)
        df['PA_VALAPR'] = pd.to_numeric(df['PA_VALAPR'], errors='coerce').fillna(0)

        # Adicionar colunas ANO e MES
        df['ANO'] = ano
        df['MES'] = mes

        # Agrupar e somar os campos
        df_agrupado = df.groupby(['PA_UFMUN', 'PA_MUNPCN', 'PA_DOCORIG', 'ANO', 'MES'], as_index=False).agg(
            PA_QTDAPR=('PA_QTDAPR', 'sum'),
            PA_VALAPR=('PA_VALAPR', 'sum')
        )

        # Definir o nome do arquivo CSV de saída
        csv_filename = os.path.join(output_dir, f"{os.path.splitext(parquet_file)[0]}.csv")

        # Salvar o DataFrame agrupado como CSV
        df_agrupado.to_csv(csv_filename, sep=';', decimal=',', index=False)

        print(f"[{datetime.now()}] Arquivo {parquet_file} processado e salvo como {csv_filename}.")
        return True
    except Exception as e:
        print(f"[{datetime.now()}] Erro ao processar {parquet_file}: {e}")
        return False
    finally:
        # Liberação de memória
        del df, df_agrupado
        gc.collect()

def main():
    arquivos_parquet = listar_arquivos_parquet(input_dir)
    total_arquivos = len(arquivos_parquet)
    print(f"[{datetime.now()}] Total de arquivos Parquet disponíveis: {total_arquivos}")

    # DataFrame para o log de processamento
    log_df = pd.DataFrame(columns=["Arquivo", "Inicio", "Fim", "Duracao (minutos)"])

    arquivos_processados = 0
    inicio_processo = datetime.now()
    for i, parquet_file in enumerate(arquivos_parquet, 1):
        inicio = datetime.now()
        sucesso = processar_arquivo_parquet(parquet_file)
        fim = datetime.now()
        duracao = (fim - inicio).total_seconds() / 60

        # Criar um DataFrame com o novo registro
        novo_registro = pd.DataFrame([{
            "Arquivo": parquet_file,
            "Inicio": inicio.strftime('%d-%m-%y %H:%M:%S'),
            "Fim": fim.strftime('%d-%m-%y %H:%M:%S'),
            "Duracao (minutos)": round(duracao, 2)
        }])

        # Verificar se novo_registro tem valores válidos
        if not novo_registro.empty and not novo_registro.isnull().values.all():
            log_df = pd.concat([log_df, novo_registro], ignore_index=True)

        if sucesso:
            arquivos_processados += 1
        print(f"Arquivos processados: {arquivos_processados} de {total_arquivos}")
        percentual_processado = (arquivos_processados / total_arquivos) * 100
        print(f"Percentual de processamento: {percentual_processado:.0f}%")

    fim_processo = datetime.now()
    duracao_processo = (fim_processo - inicio_processo).total_seconds() / 60
    print(f"[{datetime.now()}] Processo concluído em {duracao_processo:.2f} minutos.")

    # Salvar log em Excel
    log_df.to_excel(log_file, index=False)

if __name__ == "__main__":
    main()


[2024-07-29 13:57:11.166063] Total de arquivos Parquet disponíveis: 527
[2024-07-29 13:57:13.900943] Arquivo PAPE2304.parquet processado e salvo como /home/amcjr/pysus/csv_output/PAPE2304.csv.
Arquivos processados: 1 de 527
Percentual de processamento: 0%


/tmp/ipykernel_245300/2580089577.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  log_df = pd.concat([log_df, novo_registro], ignore_index=True)


[2024-07-29 13:57:15.350361] Arquivo PAMS2306.parquet processado e salvo como /home/amcjr/pysus/csv_output/PAMS2306.csv.
Arquivos processados: 2 de 527
Percentual de processamento: 0%
[2024-07-29 13:57:23.351036] Arquivo PAMG2302b.parquet processado e salvo como /home/amcjr/pysus/csv_output/PAMG2302b.csv.
Arquivos processados: 3 de 527
Percentual de processamento: 1%
[2024-07-29 13:57:25.990529] Arquivo PAAM2311.parquet processado e salvo como /home/amcjr/pysus/csv_output/PAAM2311.csv.
Arquivos processados: 4 de 527
Percentual de processamento: 1%
[2024-07-29 13:57:29.666954] Arquivo PAMG2308b.parquet processado e salvo como /home/amcjr/pysus/csv_output/PAMG2308b.csv.
Arquivos processados: 5 de 527
Percentual de processamento: 1%
[2024-07-29 13:57:37.565854] Arquivo PARS2301.parquet processado e salvo como /home/amcjr/pysus/csv_output/PARS2301.csv.
Arquivos processados: 6 de 527
Percentual de processamento: 1%
[2024-07-29 13:57:43.915274] Arquivo PAPR2302.parquet processado e salvo com